In [ ]:
import tensorflow
import torch
import numpy as np
import random,re

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

Upload the following
1. test_fullstory.txt - This file has all the stories
2. test_nrp_5con_notdep.csv - This file has training data to train Next Relationship Prediction

# Load 3 Models
## 1. BERT Classifier Model _nrp_5con_notdep

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification,TFBertModel
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,concatenate,Embedding,LSTM,Conv1D,MaxPooling1D,Flatten,Input,Bidirectional, Input
from tensorflow.keras.layers import Subtract,Lambda,Add,Conv1D
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification,TFBertModel, TFDistilBertModel, DistilBertTokenizer, TFAlbertModel , AlbertTokenizer,TFBertForQuestionAnswering
from tqdm import tqdm
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np

In [14]:
test_dataframe = pd.read_csv("data/test_nrp_5con_notdep.csv")# UPLOAD
print(test_dataframe.shape)
[test_sentences,test_labels] = [test_dataframe["text"],test_dataframe["label"]]
test_labels = test_labels.fillna("NULL")
test_sentences = list(test_sentences)
#test_dataframe

(24586, 2)


In [15]:
T=400
path = 'bert-base-cased'
model=TFBertForQuestionAnswering.from_pretrained(path)
bert_tokenizer = BertTokenizer.from_pretrained(path, do_lower_case = True)
def create_model():
  bertmodel = model#TFBertForQuestionAnswering.from_pretrained('bert-base-cased')
  In=Input(shape=(T),name="Input_Sentence",dtype=np.int64)
  o=bertmodel(In)
  o0 =  Activation("softmax",name="start")(o.start_logits)
  o1 = Activation("softmax",name="end")(o.end_logits)
  model_o=Model(inputs=In,outputs=[o0,o1])
  return model_o
model_o = create_model()
model_o.summary()

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_Sentence (InputLayer)    [(None, 400)]        0           []                               
                                                                                                  
 tf_bert_for_question_answering  TFQuestionAnswering  107721218  ['Input_Sentence[0][0]']         
 _1 (TFBertForQuestionAnswering  ModelOutput(loss=No                                              
 )                              ne, start_logits=(N                                               
                                one, 400),                                                        
                                 end_logits=(None,                                                
                                400),                                                       

In [16]:
model_o.load_weights("./models/just_weights.t5")#inplace

In [17]:
number_of_choices = [len(i.split("[SEP]")[0].split(";")) for i in test_sentences]

In [18]:
np.mean(number_of_choices)

5.6894574147889045

In [19]:
print(100*np.mean([1/i for i in number_of_choices]),100*1/np.mean(number_of_choices))

28.423172699718773 17.576368484640515


In [20]:
def ff(l):
  return bert_tokenizer.convert_tokens_to_string(l)#start end   # prod = 39.36, sum = 35.359

In [21]:
def return_nrp_prob(new_sentence):# format Null ; relations ; relation1 > text1 . relation2 > text2 .
  sample = new_sentence
  tokenized_indices = bert_tokenizer(sample.split("[SEP]")[0].strip(),sample.split("[SEP]")[1].strip(),return_tensors='tf',max_length=T,padding="max_length",add_special_tokens=True,truncation=True)["input_ids"]
  prediction = model_o.predict(tokenized_indices,verbose=0)
  tokenized_indices=tokenized_indices[0]
  #hard_pred = bert_tokenizer.convert_tokens_to_string(bert_tokenizer.convert_ids_to_tokens(tokenized_indices)[np.argmax(prediction[0][0]):np.argmax(prediction[1][0])+1]).strip()
  zipped_result  =list(zip((bert_tokenizer.convert_ids_to_tokens(tokenized_indices)[:np.where(tokenized_indices == 102)[0][0]][1:]),prediction[0][0][:np.where(tokenized_indices == 102)[0][0]][1:],prediction[1][0][:np.where(tokenized_indices == 102)[0][0]][1:]))
  useful_indices = [j for j in [-1]+[i for i in np.arange(len(zipped_result)) if zipped_result[i][0]==";"]+[len(zipped_result)]]
  possibles = [zipped_result[useful_indices[k]+1:useful_indices[k+1]] for k in np.arange(len(useful_indices)-1)]
  possibles = [(bert_tokenizer.convert_tokens_to_string([k[0] for k in j]),j[0][1],j[-1][-1]) for j in possibles]
  #soft_pred_start = [i[0] for i in possibles][np.argmax([i[1] for i in possibles])]
  #soft_pred_end = [i[0] for i in possibles][np.argmax([i[2] for i in possibles])]
  normalize_start = np.array([i[1] for i in possibles])/sum(np.array([i[1] for i in possibles]))
  return [(possibles[j][0],normalize_start[j]) for j in np.arange(len(possibles))]# just start

In [22]:
def clean(mystring):
  start = mystring.find("{")
  end = mystring.rfind("}")
  if start != -1 and end != -1:
    result = mystring[start:end+1]
    return mystring.replace(result,"").strip()
  else:
    return mystring

# Inference Time

In [25]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf
rel1_tokenizer = GPT2Tokenizer.from_pretrained('models/output_nsp_clean_parsed/',from_pt=True)
rel1_model = TFGPT2LMHeadModel.from_pretrained('models/output_nsp_clean_parsed/',pad_token_id=rel1_tokenizer.eos_token_id,from_pt=True)

2023-08-23 01:11:55.007078: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

In [31]:
import torch
from tqdm import tqdm

def return_pp(pytorch_model,tokenizer, sentence, pprint=True):#returns perplexity for prob cal
  device = 'cuda'
  encodings = tokenizer(sentence,return_tensors='pt')
  max_length = pytorch_model.config.n_positions
  stride = 1
  nlls = []
  for i in range(0, encodings.input_ids.size(1), stride):
    #print(i)
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100
    with torch.no_grad():
        outputs = pytorch_model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len
    nlls.append(neg_log_likelihood)
  ppl = torch.exp(torch.stack(nlls[1:]).sum() / end_loc)# IF stride lenght =1, keep nlls[1:]
  if pprint==True:
    print("perplexity is",ppl)
  return nlls,encodings, np.prod([np.e**(-float(i)) for i in nlls[1:]])

In [32]:
def prob_relation(text):
  return return_nrp_prob(text)

In [34]:
r = open("data/test_fullstory2.txt","r").readlines() #UPLOAD

In [37]:
story_prompts = [[i.strip() for i in j.split("<$>")[0].split(";")] for j in r]
test_line1s = [j.split("<$>")[1].split(". ")[0]+"" for j in r]

In [39]:
relations = story_prompts[0]

In [40]:
def normalize(a):
  return list(np.array(a)/sum(a))

### Load Models

In [42]:
story_prompts = [[clean(i.strip()) for i in j.split("<$>")[0].split(";")] for j in r]
test_line1s = [clean(j.split("<$>")[1].split(". ")[0]) for j in r]
ground_stories = [clean(". ".join(j.split("<$>")[1].strip().split(". "))) for j in r]

In [ ]:
def trun(lis):
  return ['{:.3g}'.format(i) for i in lis]

In [46]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

full_tokenizer = GPT2Tokenizer.from_pretrained('models/output_fullstory',from_pt=True)
full_model= TFGPT2LMHeadModel.from_pretrained('models/output_fullstory',pad_token_id=full_tokenizer.eos_token_id,from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [48]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

con_tokenizer = GPT2Tokenizer.from_pretrained('models/output_nsp_clean_parsed/',from_pt=True)
con_model= TFGPT2LMHeadModel.from_pretrained('models/output_nsp_clean_parsed/',pad_token_id=con_tokenizer.eos_token_id,from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [49]:
def bracket_clean(text):
    return re.sub("[\(\[\{].*?[\}\)\]]", "", text).replace("{","").replace("}","").replace("[","").replace("]","").replace("(","").replace(")","").replace("  "," ").strip()

In [51]:
nrp_sentence="NULL ; Amy positive Wisk [SEP] Amy is not going to go home."
all_relations_with_null=["Amy positive Wisk","NULL"]

In [ ]:
"""
#rough
prob=np.array(list(zip(*return_nrp_prob(nrp_sentence)))[1])
prob=prob/sum(prob)
choosen_relation = np.random.choice(list(zip(*return_nrp_prob(nrp_sentence)))[0],p=prob)
prob, all_relations_with_null#.("Rachel negative Locke")
all_relations_with_null, int(2.75)
[i.capitalize()  if i not in ["negative","positive","neutral"] else i for i in choosen_relation.split()]
choosen_relation
ct=0
for j in [" ".join(i) for i in story_prompts]:
    if " he " in j or " she " in j or " his " in j or " they " in j:
        ct=ct+1
ct*100/len(story_prompts)
choosen_relation="Null"
if choosen_relation.lower()=="null":
    choosen_relation="NULL"
choosen_relation
return_nrp_prob(nrp_sentence)
"AAAA".capitalize()"""

In [60]:
def cap_string(choosen_relation):
    if choosen_relation.lower()=="null":
        choosen_relation="NULL"
    else:
        choosen_relation = " ".join([i.capitalize()  if i not in ["negative","positive","neutral"] else i for i in choosen_relation.split()])
    return choosen_relation
cap_string("Null"),cap_string("null"),cap_string("rachel positive locke"),cap_string("Rachel positive Locke")

('NULL', 'NULL', 'Rachel positive Locke', 'Rachel positive Locke')

In [61]:
ct=0
for i in story_prompts:
    for j in i:
        if "'" in j:
            #print(i)#$s = re.sub(r'\W+', '', input)
            ct=ct+1
print(ct*100/len(story_prompts))# % of noisy realaitonshops

0.4139834406623735


### Custom Relationships and First Line to create a story

In [ ]:
prints=True
#for INDEX in tqdm(np.arange(1)):#4 hours for 100 ############################################
start_sent = "Amy goes to the market to buy vegetables"# no fullstop
all_relations = ["Amy negative Wisk","Amy positive Julia"]
#################################################################
all_relations = [re.sub('[^ 0-9a-zA-Z]+', '',i) for i in all_relations]# remove qoutes and fullstops from realtioans
all_relations_with_null = ["NULL"] + all_relations
story_so_far = [con_tokenizer.decode(con_tokenizer.encode(start_sent)).strip() + "."]# all sentences should have "." no space
relations_so_far = ["NULL"]
story_pairs=[]
print("Seed Story:",story_so_far)  if prints==True else None
print("Relation Set:",all_relations_with_null)  if prints==True else None
K=5
for counter in np.arange(10):
    #print("\n")
    nrp_sentence = " ; ".join(all_relations_with_null) + " [SEP] " + "; ".join([relations_so_far[i].lower()+" > "+ story_so_far[i].lower() for i in np.arange(len(story_so_far))])#"SEP capital"
    #print("NRP_sentence",nrp_sentence)
    #output = return_nrp_prob(nrp_sentence)
    prob=np.array(list(zip(*return_nrp_prob(nrp_sentence)))[1])
    #prob = np.array([1/len(prob)]*len(prob))### uniformication
    prob=prob/sum(prob)
    choosen_relation = cap_string(np.random.choice(list(zip(*return_nrp_prob(nrp_sentence)))[0],p=prob))
    print("o",choosen_relation)  if prints==True else None
    # diversification#########
    choosen_relation=np.random.choice(["NULL",choosen_relation],p=[0.4,0.6])#0.4,0.6
    relations_so_far.append(choosen_relation)
    if choosen_relation.lower()!="null":
        if relations_so_far.count(choosen_relation) > max(int(10/len(all_relations_with_null))-1,2):#at least use r twice before dropping
            all_relations_with_null.remove(choosen_relation) if choosen_relation in all_relations_with_null else None# if because error handling 
    #######################################
    print("CHOOSEN RELATION: ",choosen_relation)  if prints==True else None
    if choosen_relation.lower()!="null":
      input_context = " ".join(story_so_far)+ " <%> " + choosen_relation + " <$>"# DO NOT END with SPACE HERE no need for :k
      input_ids = con_tokenizer.encode( input_context, return_tensors='tf')
      print("INPUT TO THE MODEL:",input_context)  if prints==True else None # verify the input condeitional model is recviing
      output = con_model.generate(input_ids, max_length=input_ids.shape[1]+100, num_beams=5,  no_repeat_ngram_size=3)#,early_stopping==True)
      choosen_relation_sentence = con_tokenizer.decode(output[0], skip_special_tokens=False)
      print("TOTAL OUTPUT      :",choosen_relation_sentence)  if prints==True else None
      new_sentences = choosen_relation_sentence.split("<$> ")[1].strip().split(". ")[:2]#.strip()+"."
      new_sentence = ". ".join(new_sentences) + "."
      new_sentence = bracket_clean(new_sentence)
      print("NEW SENTENCE:",new_sentence)  if prints==True else None
      story_so_far.append(new_sentence)
      relations_so_far.append(choosen_relation)
      if "<EOS>" in new_sentence:
        break
      #print(story_so_far)
    if choosen_relation.lower()=="null":
      input_context = " ; ".join(all_relations) + " <$> " + " ".join(story_so_far).strip()# DO NOT END with SPACE HERE
      input_ids = full_tokenizer.encode(input_context, return_tensors='tf')
      print("INPUT TO THE MODEL:",input_context)  if prints==True else None # verify the input condeitional model is recviing
      output = full_model.generate(input_ids, max_length=input_ids.shape[1]+100, num_beams=5,  no_repeat_ngram_size=3)#,early_stopping==True)
      choosen_relation_sentence = full_tokenizer.decode(output[0], skip_special_tokens=False)
      print("TOTAL OUTPUT      :",choosen_relation_sentence)  if prints==True else None
      input_context = con_tokenizer.decode(con_tokenizer.encode(input_context)).strip()
      new_sentences = choosen_relation_sentence.split(input_context)[1].strip().split(". ")[:5]# before splitting, need to make sure input_context exists in generated
      if len(new_sentences)>1: # error handling
          new_sentence = new_sentences[0]+". "+new_sentences[1]+"."
      else:
          new_sentence = new_sentences[0]
      new_sentence = bracket_clean(new_sentence)
      print("NEW SENTENCES:",new_sentence)  if prints==True else None
      story_so_far.append(new_sentence)
      relations_so_far.append(choosen_relation)
      #print(story_so_far)
      if "\n" in new_sentence:
        break

### Running on Complete Set

In [64]:
silver_full_stories=[]#
triad_relations=[]
silver_full_relations=[]
Triad=True
prints=False
for INDEX in tqdm(np.arange(1000)):#4 hours for 100
  all_relations = story_prompts[INDEX].copy()
  all_relations = [re.sub('[^ 0-9a-zA-Z]+', '',i) for i in all_relations]# remove qoutes and fullstops from realtioans
  all_relations_with_null = ["NULL"] + all_relations
  story_so_far = [con_tokenizer.decode(con_tokenizer.encode(test_line1s[INDEX])).strip() + "."]# all sentences should have "." no space
  relations_so_far = ["NULL"]
  story_pairs=[]
  print("Seed Story:",story_so_far)  if prints==True else None
  print("Relation Set:",all_relations_with_null)  if prints==True else None
  K=5
  for counter in np.arange(10):
    #print("\n")
    nrp_sentence = " ; ".join(all_relations_with_null) + " [SEP] " + "; ".join([relations_so_far[i].lower()+" > "+ story_so_far[i].lower() for i in np.arange(len(story_so_far))])#"SEP capital"
    #print("NRP_sentence",nrp_sentence)
    #output = return_nrp_prob(nrp_sentence)
    prob=np.array(list(zip(*return_nrp_prob(nrp_sentence)))[1])
    #prob = np.array([1/len(prob)]*len(prob))### uniformication
    prob=prob/sum(prob)
    choosen_relation = cap_string(np.random.choice(list(zip(*return_nrp_prob(nrp_sentence)))[0],p=prob))
    print("o",choosen_relation)  if prints==True else None
    triad_relations.append(choosen_relation)
    if Triad==False:
        ###########diversification#########
        choosen_relation=np.random.choice(["NULL",choosen_relation],p=[0.4,0.6])#0.4,0.6
        relations_so_far.append(choosen_relation)
        if choosen_relation.lower()!="null":
            if relations_so_far.count(choosen_relation) > max(int(10/len(all_relations_with_null))-1,2):#at least use r twice before dropping
                all_relations_with_null.remove(choosen_relation) if choosen_relation in all_relations_with_null else None# if because error handling 
        #######################################
    print("CHOOSEN RELATION: ",choosen_relation)  if prints==True else None
    if choosen_relation.lower()!="null":
      input_context = " ".join(story_so_far)+ " <%> " + choosen_relation + " <$>"# DO NOT END with SPACE HERE no need for :k
      input_ids = con_tokenizer.encode( input_context, return_tensors='tf')
      print("INPUT TO THE MODEL:",input_context)  if prints==True else None # verify the input condeitional model is recviing
      output = con_model.generate(input_ids, max_length=input_ids.shape[1]+100, num_beams=5,  no_repeat_ngram_size=3)#,early_stopping==True)
      choosen_relation_sentence = con_tokenizer.decode(output[0], skip_special_tokens=False)
      print("TOTAL OUTPUT      :",choosen_relation_sentence)  if prints==True else None
      new_sentences = choosen_relation_sentence.split("<$> ")[1].strip().split(". ")[:2]#.strip()+"."
      new_sentence = ". ".join(new_sentences) + "."
      new_sentence = bracket_clean(new_sentence)
      print("NEW SENTENCE:",new_sentence)  if prints==True else None
      story_so_far.append(new_sentence)
      relations_so_far.append(choosen_relation)
      if "<EOS>" in new_sentence:
        break
      #print(story_so_far)
    if choosen_relation.lower()=="null":
      input_context = " ; ".join(all_relations) + " <$> " + " ".join(story_so_far).strip()# DO NOT END with SPACE HERE
      input_ids = full_tokenizer.encode(input_context, return_tensors='tf')
      print("INPUT TO THE MODEL:",input_context)  if prints==True else None # verify the input condeitional model is recviing
      output = full_model.generate(input_ids, max_length=input_ids.shape[1]+100, num_beams=5,  no_repeat_ngram_size=3)#,early_stopping==True)
      choosen_relation_sentence = full_tokenizer.decode(output[0], skip_special_tokens=False)
      print("TOTAL OUTPUT      :",choosen_relation_sentence)  if prints==True else None
      input_context = con_tokenizer.decode(con_tokenizer.encode(input_context)).strip()
      new_sentences = choosen_relation_sentence.split(input_context)[1].strip().split(". ")[:5]# before splitting, need to make sure input_context exists in generated
      if len(new_sentences)>1: # error handling
          new_sentence = new_sentences[0]+". "+new_sentences[1]+"."
      else:
          new_sentence = new_sentences[0]
      new_sentence = bracket_clean(new_sentence)
      print("NEW SENTENCES:",new_sentence)  if prints==True else None
      story_so_far.append(new_sentence)
      relations_so_far.append(choosen_relation)
      #print(story_so_far)
      if "\n" in new_sentence:
        break
  silver_full_stories.append(story_so_far)
  silver_full_relations.append(relations_so_far)

  0%|                                                                                                 | 0/100 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if s

KeyboardInterrupt: 